In [2]:
import pandas as pd
from googletrans import Translator
import re
from nltk.tokenize import word_tokenize

In [33]:
# Read the dataset
data = pd.read_excel('datasets/2018-E-c-Ar-train.xlsx')


In [34]:
data_raw = data.copy()

In [35]:

# Regular expression pattern to match any non-Arabic characters
english_pattern = r'\b[A-Za-z]+\b'

# Regular expression pattern to match any digits
digits_pattern = r'\d+'


In [36]:
def count_english_words(data):
    # Tokenize the text into words
    tokens = data['Tweet'].apply(word_tokenize)

    # Filter out non-Arabic words using the regular expression
    english_words = tokens.apply(lambda words: [word for word in words if re.match(english_pattern, word)])

    # Count the occurrences of each Arabic word
    english_words = pd.Series([word for words in english_words for word in words]).value_counts()

    # Print the top 10 most frequent Arabic words
    print(english_words.head(10))



In [37]:
count_english_words(data)

AkramHosny         5
feeling            3
nadinenjeim        2
mshinqiti          2
D                  2
lt                 2
nowaaaash          1
Idgix              1
Nadaabdulwahab     1
AsmaAlmuhannadi    1
dtype: int64


In [38]:
data['Tweet'] = data['Tweet'].apply(lambda x: re.sub(english_pattern, '', x))

In [39]:
count_english_words(data)

Series([], dtype: int64)


C:\Users\pc\AppData\Local\Temp\ipykernel_17396\1444391590.py:9: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  english_words = pd.Series([word for words in english_words for word in words]).value_counts()


In [40]:
def count_digits(data):
    # Apply the regular expression pattern to each row of the "tweet" column and count the digits
    digit_counts = data['Tweet'].apply(lambda x: len(re.findall(digits_pattern, x))).sum()

    # Print the total count of digits in all tweets
    print('Total digits count:', digit_counts)


In [41]:
count_digits(data)

Total digits count: 331


In [42]:
# Apply the regular expression pattern to each row of the "tweet" column
data['Tweet'] = data['Tweet'].apply(lambda x: re.sub(digits_pattern, '', x))

In [43]:
count_digits(data)

Total digits count: 0


In [44]:
data.head()

,ID,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-Ar-00259,ظلينا نتكلم ساعات ساعات رتبت فيها نفسي وبكيت ف...,1,0,0,1,0,0,0,1,1,0,0
1,2018-Ar-02696,كل سنه وانتي بخير ياقلبي وكل سنه وانتي سعيده ي...,0,0,0,0,1,1,1,0,0,0,0
2,2018-Ar-03596,البسطاء يمتلكون أرواح نادره جداتجدهم بمظهر متو...,0,0,0,0,0,1,1,0,0,0,0
3,2018-Ar-02999,مومعقول اللي قاعد يصير فيني هالايام يارب ماينت...,0,0,0,1,0,0,0,0,0,0,0
4,2018-Ar-02716,انا اكثر شخص متناقض بداخلي حب وكره وامل وقنوط ...,1,0,0,0,0,0,0,0,1,0,0
